In [ ]:
#Script does the following:
#Reads the content of two Word files
#Compares them line by line
#Writes the differences to a text file.

import docx
import difflib

def read_docx(file_path):
    doc = docx.Document(file_path)
    paragraphs = [para.text for para in doc.paragraphs]
    table_texts = []
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                table_texts.append(cell.text)
    return paragraphs, table_texts

def highlight_changes(doc, diff):
    for line in diff:
        if line.startswith('- '):
            highlight_text(doc, line[2:], 'FF0000')  # Red for deletions
        elif line.startswith('+ '):
            highlight_text(doc, line[2:], '00FF00')  # Green for additions

def highlight_text(doc, text, color):
    for para in doc.paragraphs:
        if text in para.text:
            for run in para.runs:
                if text in run.text:
                    run.font.highlight_color = docx.enum.text.WD_COLOR_INDEX.YELLOW
                    run.font.color.rgb = docx.shared.RGBColor.from_string(color)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if text in cell.text:
                    for para in cell.paragraphs:
                        for run in para.runs:
                            if text in run.text:
                                run.font.highlight_color = docx.enum.text.WD_COLOR_INDEX.YELLOW
                                run.font.color.rgb = docx.shared.RGBColor.from_string(color)

def write_diff_to_file(diff, output_file):
    with open(output_file, 'w') as f:
        for line in diff:
            f.write(line + '\n')

def main(file1, file2, output_file):
    doc1 = docx.Document(file1)
    doc2 = docx.Document(file2)
    
    text1_paragraphs, text1_tables = read_docx(file1)
    text2_paragraphs, text2_tables = read_docx(file2)

    text1 = text1_paragraphs + text1_tables
    text2 = text2_paragraphs + text2_tables

    diff = difflib.unified_diff(text1, text2, lineterm='')
    highlight_changes(doc1, diff)
    
    try:
        doc1.save(output_file)
        print(f"Updated document saved as {output_file}")
    except Exception as e:
        print(f"Error saving updated document: {e}")


#Customize the file paths (file1, file2, and output_file) as needed.
if __name__ == "__main__":
    file1 = 'Test_SOP.docx'
    file2 = 'Updated_Test_SOP.docx'
    output_file = 'highlighted_differences.docx'
    main(file1, file2, output_file)

